# 📌 Implementazione di una Pipeline RAG con ChromaDB e Ollama

Questo notebook illustra passo dopo passo la costruzione di una pipeline **Retrieval-Augmented Generation (RAG)** usando **ChromaDB** per la ricerca semantica e **Ollama** per la generazione del testo.

Ogni sezione include codice Python e una spiegazione dettagliata.

---

## 1. Importazione delle librerie

### 📌 Analisi riga per riga
- `OllamaEmbeddings`: Crea embedding numerici per i documenti e le query.
- `OllamaLLM`: Interfaccia per interrogare il modello LLM di Ollama.
- `chromadb`: Libreria per il database vettoriale ChromaDB.
- `os`: Permette di gestire i percorsi dei file e la persistenza del database.

### 📌 Ruolo nel sistema RAG
- **LangChain-Ollama** genera embedding e risposte testuali.
- **ChromaDB** archivia gli embedding e supporta la ricerca vettoriale.
- **OS** è necessario per salvare il database in locale.

---

## 2. Definizione del modello LLM

### 📌 Ruolo nel sistema RAG
- Indica il modello LLM che verrà usato **sia per il calcolo degli embedding che per la generazione delle risposte**.

---

## 3. Configurazione del database ChromaDB

### 📌 Cosa fa questa riga?
- Inizializza **ChromaDB con storage persistente** in `chroma_db/`.
- Se `chroma_db/` non esiste, lo crea.
- I dati verranno memorizzati in SQLite (`chroma.sqlite3`) e in **file binari separati per gli embedding**.

### 📌 Struttura fisica generata
| **File/Cartella** | **Descrizione** |
|------------------|----------------|
| `chroma.sqlite3` | Database SQLite che contiene metadati e riferimenti agli embedding. |
| `UUID/` | Cartella con ID univoco che contiene gli embedding veri e propri. |
| `data_level0.bin` | File binario che contiene gli embedding memorizzati. |
| `link_lists.bin` | Struttura di indicizzazione per la ricerca ANN. |

---

## 4. Definizione della funzione di embedding personalizzata

### 📌 Cosa fa questa classe?
- Converte i documenti in **vettori numerici**.
- ChromaDB **non genera embedding**, quindi ha bisogno di questa funzione per accettare testo e restituire vettori.

### 📌 Flusso dei dati
1. Un documento (`input`) viene ricevuto.
2. Viene convertito in embedding da `OllamaEmbeddings`.
3. L’output è una lista di vettori che possono essere memorizzati in ChromaDB.

---

## 5. Creazione della funzione di embedding

### 📌 Cosa fa questa riga?
- Inizializza **OllamaEmbeddings** per generare i vettori con `llama3.2`.
- Usa `http://localhost:11434` per comunicare con il server Ollama.

### 📌 Perché è importante?
- **ChromaDB ha bisogno di un metodo esterno** per ottenere embedding.
- Ollama genera i vettori che poi vengono salvati in `data_level0.bin`.

---

## 6. Creazione della collezione in ChromaDB

### 📌 Cosa fa questa riga?
- Crea una **collezione** chiamata `"rag_collection_demo_1"`.
- Se esiste già, la recupera.
- Associa la funzione di embedding (`embedding_function=embedding`).

### 📌 Struttura logica della collezione
| **Campo** | **Contenuto** |
|-----------|--------------|
| `id` | ID univoco del documento. |
| `embedding` | Vettore numerico associato al documento. |
| `document` | Testo originale. |
| `metadata` | Informazioni aggiuntive (es. autore, fonte). |

---

## 7. Funzione per aggiungere documenti

### 📌 Cosa fa questa funzione?
- Riceve **documenti testuali** e **ID univoci**.
- Genera embedding e li memorizza nella collezione.

---

## 8. Aggiunta di documenti di esempio

### 📌 Cosa succede qui?
1. I documenti vengono trasformati in embedding.
2. Gli embedding vengono salvati in `data_level0.bin`.
3. `chroma.sqlite3` registra i metadati.

---

## 9. Funzione per interrogare ChromaDB

### 📌 Cosa fa questa funzione?
- Converte la query in un embedding.
- Cerca i documenti **più simili** con **ANN + Cosine Similarity**.
- Restituisce i documenti corrispondenti.

---

## 10. Funzione per interrogare Ollama

### 📌 Cosa fa questa funzione?
- Invia una richiesta a Ollama.
- Restituisce una risposta generata dal modello.

---

## 11. Pipeline RAG

### 📌 Fasi della pipeline
1. **Recupera documenti** da ChromaDB.
2. **Costruisce un prompt migliorato**.
3. **Interroga Ollama per generare la risposta**.

---

## 12. Test della pipeline

### 📌 Cosa fa?
- Esegue la pipeline completa su una query di esempio.

---

# 🎯 Conclusione
Questo notebook ha mostrato come implementare un sistema **RAG** utilizzando **ChromaDB per la memorizzazione e il retrieval di documenti vettorializzati** e **Ollama per la generazione di risposte basate su contesto estratto**.

Se vuoi eseguire il codice, assicurati di avere un server **Ollama attivo su `http://localhost:11434`** e che **ChromaDB sia correttamente installato**. 🚀


In [13]:
from langchain_ollama import OllamaEmbeddings, OllamaLLM
import chromadb
import os

In [14]:
llm_model = "llama3.2"

In [15]:
chroma_client = chromadb.PersistentClient(path=os.path.join(os.getcwd(), "chroma_db"))

In [16]:
class ChromaDBEmbeddingFunction:
    """
    Funzione personalizzata per generare embedding con Ollama.
    """
    def __init__(self, langchain_embeddings):
        self.langchain_embeddings = langchain_embeddings

    def __call__(self, input):
        # Se il testo è una stringa singola, lo converte in lista
        if isinstance(input, str):
            input = [input]
        return self.langchain_embeddings.embed_documents(input)

In [17]:
embedding = ChromaDBEmbeddingFunction(
    OllamaEmbeddings(
        model=llm_model,
        base_url="http://localhost:11434"
    )
)

In [18]:
collection_name = "rag_collection_demo_1"
collection = chroma_client.get_or_create_collection(
    name=collection_name,
    metadata={"description": "A collection for RAG with Ollama - Demo1"},
    embedding_function=embedding
)

In [19]:
def add_documents_to_collection(documents, ids):
    """
    Aggiunge documenti alla collezione ChromaDB.
    """
    collection.add(
        documents=documents,
        ids=ids
    )

In [20]:
documents = [
    "Artificial intelligence is the simulation of human intelligence processes by machines.",
    "Python is a programming language that lets you work quickly and integrate systems more effectively.",
    "ChromaDB is a vector database designed for AI applications."
]
doc_ids = ["doc1", "doc2", "doc3"]

add_documents_to_collection(documents, doc_ids)

Add of existing embedding ID: doc1
Add of existing embedding ID: doc2
Add of existing embedding ID: doc3
Insert of existing embedding ID: doc1
Insert of existing embedding ID: doc2
Insert of existing embedding ID: doc3


In [21]:
def query_chromadb(query_text, n_results=1):
    """
    Cerca documenti pertinenti nella collezione ChromaDB.
    """
    results = collection.query(
        query_texts=[query_text],
        n_results=n_results
    )
    return results["documents"], results["metadatas"]

In [22]:
def query_ollama(prompt):
    """
    Invia una query a Ollama e restituisce la risposta.
    """
    llm = OllamaLLM(model=llm_model)
    return llm.invoke(prompt)

In [23]:
def rag_pipeline(query_text):
    """
    Pipeline RAG: recupera documenti e genera una risposta.
    """
    retrieved_docs, metadata = query_chromadb(query_text)
    context = " ".join(retrieved_docs[0]) if retrieved_docs else "No relevant documents found."

    augmented_prompt = f"Context: {context}\n\nQuestion: {query_text}\nAnswer:"
    print("######## Augmented Prompt ########")
    print(augmented_prompt)

    response = query_ollama(augmented_prompt)
    return response

In [24]:
query = "What is an LLM?"
response = rag_pipeline(query)
print("######## Response from LLM ########\n", response)

######## Augmented Prompt ########
Context: Artificial intelligence is the simulation of human intelligence processes by machines.

Question: What is an LLM?
Answer:
######## Response from LLM ########
 A Large Language Model (LLM) is a type of artificial intelligence (AI) that uses machine learning techniques to process and understand natural language. It's trained on massive amounts of text data, enabling it to generate human-like responses to a wide range of questions and tasks.
